# pySwipe

> Notebook authors: Spencer M. Hatch

> Package authors: Spencer M. Hatch, Karl M. Laundal

## Resources

Python package: `pyswipe`  
[Code](https://github.com/Dartspacephysiker/pyswipe) 

In [ ]:
%load_ext watermark
%watermark -i -v -p pyswipe,numpy,matplotlib

## Introduction


In this notebook we will use [pySwipe](https://github.com/Dartspacephysiker/pyswipe) to make some simple summary plots of the high-latitude ionospheric electric potential $\Phi$, electric field $\mathbf{E} = -\nabla \Phi$, height-integrated electromagnetic work $\mathbf{J}\cdot\mathbf{E}$, and Hall and Pedersen conductances $\Sigma_H$ and $\Sigma_P$.

Let's start by importing pyswipe and initializing a SWIPE object.

In [ ]:
import numpy as np 
from matplotlib import pyplot as plt

from pyswipe import SWIPE

#Define solar wind, IMF, and season
vsw = 400 # solar wind speed [km/s]
by = 3 #IMF By in GSM coordinates [nT]
bz = -3 #IMF Bz in GSM coordinates [nT]
dptilt = 0. #Dipole tilt [deg]
f107val = 120 #F10.7 [s.f.u., typical values are 60–80 and 140–220 during solar min and max, respectively]
minlat = 45 #minimum latitude for SWIPE model grid
this = SWIPE(vsw,by,bz,dptilt,f107val,
             minlat=minlat)

## Electric potential and electric field 

Let's first use this object to produce a quick plot of the electric potential in each hemisphere. 
* The `plot_potential()` method uses output from the `get_potential()` method and some plotting utilities included in pyswipe to produce this plot.

In [ ]:
this.plot_potential()

# We could instead use the following if we wanted to get the value of the potential at each of these locations
#mlats, mlts = this.plotgrid_scalar
#phin, phis = np.split(this.get_potential(), 2)
#phin = phin - np.median(phin)
#phis = phis - np.median(phis)

* The potential distributions in the Northern and Southern Hemisphere for the conditions selected by the user are shown in the left and right panels. 
* The gray quivers/arrows in each panel show the electric field. 
* The cross-polar cap potential (i.e., maximum potential - minimum potential) is shown at bottom right in each panel. 
* The contour spacing is 5 kV.

## Height-integrated electromagnetic work $\mathbf{J}\cdot\mathbf{E}$

Let us now plot height-integrated electromagnetic work.

In [ ]:
this.plot_emwork()

## Hall and Pedersen conductances

Let's now have a look at the Hall and Pedersen conductances.

In [ ]:
this.plot_conductance(vmin=0.)

* Hall and Pedersen conductance distributions are respectively shown in the top and bottom rows
* Q: "What's up with the gray background?"
    * A: The gray background indicates a location where one of the following conditions is broken:
        * 0.05 mho $< \Sigma_H < 100$ mho
        * $\mathbf{J}\cdot\mathbf{E} < $ 0.5 mW/m²
    * When one or both of these conditions are not met, it means that output from the Swipe model may not be consistent with the assumption that the neutral wind is zero in the Earth's rotating frame of reference. Why? Because if the neutral wind truly is zero in the Earth's rotating frame of reference, then in that frame of reference $\mathbf{J}\cdot\mathbf{E} > 0$.
    * Another possibility: When these conditions aren't met, it might mean that it doesn't make sense to speak of "average" ionospheric electrodynamics for the given solar wind conditions, or it might just be an indication that some other aspect of the Swipe model is not perfect, like our choice of a spherical harmonic representation of the electric potential and current systems)
    * So! Where model conductances are shown, they are *possibly* consistent with the assumption that the neutral wind is zero in the Earth's rotating frame of reference.

## Model values at user-specified coordinates

We can also get the potential, electric field, and height-integrated electromagnetic work at selected locations.

Let's examine the noon-midnight meridian.

In [ ]:
mlats = np.arange(45,135.1,0.25)
mlts = np.ones_like(mlats)*12
flip = mlats > 90
mlatsnowrap = mlats.copy()
mlats[flip] = 180-mlats[flip]
mlts[flip] = (mlts[flip]+12)%24
#print(mlats)
#print(mlts)
pot = this.get_potential(mlats,mlts,grid=False)
Ed1, Ed2 = this.get_efield_MA(mlats,mlts,grid=False)
emwork = this.get_emwork(mlats,mlts,grid=False)

fig, axes = plt.subplots(4,1,figsize=(10,10),sharex=True)
fig.suptitle("Noon-midnight meridian\n("+r"$v_{sw}=$"+str(vsw)+r", $B_y=$"+str(by)+r", $B_z=$"+str(bz)+r", $\Psi=$"+str(dptilt)+")",fontsize=15)

tickmlats = np.arange(50,131,10)
tickmlatswrap = tickmlats.copy()
tickmlatswrap[tickmlats > 90] = 180-tickmlatswrap[tickmlats > 90]

#potential
plt.sca(axes[0])
plt.plot(mlatsnowrap,pot)
plt.xticks(ticks=tickmlats,labels=tickmlatswrap)
#plt.title("Potential")
plt.ylabel("Potential [kV]")

#E-field
plt.sca(axes[1])
plt.plot(mlatsnowrap,Ed1)
plt.ylabel("$E_{d1}$ [mV/m]")

plt.sca(axes[2])
plt.plot(mlatsnowrap,Ed2)
plt.ylabel("$E_{d2}$ [mV/m]")

#EM work
plt.sca(axes[3])
plt.plot(mlatsnowrap,emwork)
plt.ylabel("EM work [mW/m²]")

plt.xlabel("MLat [deg]")

#Mark 90° MLat
for ax in axes:
    ax.axvline(90,linestyle='--',color='gray',zorder=-10)
    
plt.tight_layout()

* First panel: Potential
* Second panel: Electric field in the (approximately) magnetic east-west direction
* Third panel: Electric field in the (approximately) magnetic north-south direction
* Fourth panel: Height-integrated electromagnetic work

The discontinuity in the electric field components at 90° MLat is a reflection of the fact that the angular basis vectors of a spherical coordinate system are discontinuous at the poles. 

Let's make a figure in the same layout as the one above, but this time showing these quantities along the dawn-dusk meridian

In [ ]:
mlats = np.arange(45,135.1,0.25)
mlts = np.ones_like(mlats)*6
flip = mlats > 90
mlatsnowrap = mlats.copy()
mlats[flip] = 180-mlats[flip]
mlts[flip] = (mlts[flip]+12)%24
#print(mlats)
#print(mlts)
pot = this.get_potential(mlats,mlts,grid=False)
Ed1, Ed2 = this.get_efield_MA(mlats,mlts,grid=False)
emwork = this.get_emwork(mlats,mlts,grid=False)

fig, axes = plt.subplots(4,1,figsize=(10,10),sharex=True)
fig.suptitle("Dawn-dusk meridian\n("+r"$v_{sw}=$"+str(vsw)+r", $B_y=$"+str(by)+r", $B_z=$"+str(bz)+r", $\Psi=$"+str(dptilt)+")",fontsize=15)

tickmlats = np.arange(50,131,10)
tickmlatswrap = tickmlats.copy()
tickmlatswrap[tickmlats > 90] = 180-tickmlatswrap[tickmlats > 90]

#potential
plt.sca(axes[0])
plt.plot(mlatsnowrap,pot)
plt.xticks(ticks=tickmlats,labels=tickmlatswrap)
#plt.title("Potential")
plt.ylabel("Potential [kV]")

#E-field
plt.sca(axes[1])
plt.plot(mlatsnowrap,Ed1)
plt.ylabel("$E_{d1}$ [mV/m]")

plt.sca(axes[2])
plt.plot(mlatsnowrap,Ed2)
plt.ylabel("$E_{d2}$ [mV/m]")

#EM work
plt.sca(axes[3])
plt.plot(mlatsnowrap,emwork)
plt.ylabel("EM work [mW/m²]")

plt.xlabel("MLat [deg]")

#Mark 90° MLat
for ax in axes:
    ax.axvline(90,linestyle='--',color='gray',zorder=-10)
    
plt.tight_layout()

## Along user-specified positions

In this notebook we will use [pySwipe](https://github.com/Dartspacephysiker/pyswipe) to make plots of the Swipe high-latitude ionospheric electric potential $\Phi$, electric field $\mathbf{E} = -\nabla \Phi$, and height-integrated electromagnetic work $\mathbf{J}\cdot\mathbf{E}$ along the noon-midnight and dawn-dusk meridians.

Let's start by importing pyswipe and initializing a SWIPE object.

In [ ]:
import numpy as np 
from matplotlib import pyplot as plt

from pyswipe import SWIPE

#Define solar wind, IMF, and season
vsw = 400 # solar wind speed [km/s]
by = 3 #IMF By in GSM coordinates [nT]
bz = -3 #IMF Bz in GSM coordinates [nT]
dptilt = 0. #Dipole tilt [deg]
f107val = 120 #F10.7 [s.f.u., typical values are 60–80 and 140–220 during solar min and max, respectively]
minlat = 45 #minimum latitude for SWIPE model grid
this = SWIPE(vsw,by,bz,dptilt,f107val,
             minlat=minlat)

## Model values at user-specified coordinates

Let's now get the potential, electric field, and height-integrated electromagnetic work along the noon-midnight meridian.

In [ ]:
mlats = np.arange(45,135.1,0.25)
mlts = np.ones_like(mlats)*12
flip = mlats > 90
mlatsnowrap = mlats.copy()
mlats[flip] = 180-mlats[flip]
mlts[flip] = (mlts[flip]+12)%24
#print(mlats)
#print(mlts)
pot = this.get_potential(mlats,mlts,grid=False)
Ed1, Ed2 = this.get_efield_MA(mlats,mlts,grid=False)
emwork = this.get_emwork(mlats,mlts,grid=False)
sh, sp, mask = this.get_conductances(mlats,mlts)
sh = np.ma.masked_array(sh,mask)
sp = np.ma.masked_array(sp,mask)
conmax = np.max([np.max(sh),np.max(sp)])

fig, axes = plt.subplots(6,1,figsize=(10,10),sharex=True)
fig.suptitle("Noon-midnight meridian\n("+r"$v_{sw}=$"+str(vsw)+r", $B_y=$"+str(by)+r", $B_z=$"+str(bz)+r", $\Psi=$"+str(dptilt)+")",fontsize=15)

tickmlats = np.arange(50,131,10)
tickmlatswrap = tickmlats.copy()
tickmlatswrap[tickmlats > 90] = 180-tickmlatswrap[tickmlats > 90]

#potential
plt.sca(axes[0])
plt.plot(mlatsnowrap,pot)
plt.xticks(ticks=tickmlats,labels=tickmlatswrap)
#plt.title("Potential")
plt.ylabel("Potential [kV]")

#E-field
plt.sca(axes[1])
plt.plot(mlatsnowrap,Ed1)
plt.ylabel("$E_{d1}$ [mV/m]")

plt.sca(axes[2])
plt.plot(mlatsnowrap,Ed2)
plt.ylabel("$E_{d2}$ [mV/m]")

#EM work
plt.sca(axes[3])
plt.plot(mlatsnowrap,emwork)
plt.ylabel("EM work [mW/m²]")

#Conductances
plt.sca(axes[4])
plt.plot(mlatsnowrap,sh)
plt.ylabel("$\Sigma_H$ [mho]")
plt.ylim((0,conmax))

plt.sca(axes[5])
plt.plot(mlatsnowrap,sp)
plt.ylabel("$\Sigma_P$ [mho]")
plt.ylim((0,conmax))

plt.xlabel("MLat [deg]")

#Mark 90° MLat
for ax in axes:
    ax.axvline(90,linestyle='--',color='gray',zorder=-10)
    
plt.tight_layout()

* First panel: Potential
* Second panel: Electric field in the (approximately) magnetic east-west direction
* Third panel: Electric field in the (approximately) magnetic north-south direction
* Fourth panel: Height-integrated electromagnetic work
* Fifth and six panels: Hall and Pedersen conductances, with conductance estimates *masked* where these estimates are likely to be bad. (See `pyswipe_introduction.ipynb`.)

The discontinuity in the electric field components at 90° MLat is a reflection of the fact that the angular basis vectors of a spherical coordinate system are discontinuous at the poles. 

Let's make a figure in the same layout as the one above, but this time showing these quantities along the dawn-dusk meridian

In [ ]:
mlats = np.arange(45,135.1,0.25)
mlts = np.ones_like(mlats)*6
flip = mlats > 90
mlatsnowrap = mlats.copy()
mlats[flip] = 180-mlats[flip]
mlts[flip] = (mlts[flip]+12)%24
#print(mlats)
#print(mlts)
pot = this.get_potential(mlats,mlts,grid=False)
Ed1, Ed2 = this.get_efield_MA(mlats,mlts,grid=False)
emwork = this.get_emwork(mlats,mlts,grid=False)
sh, sp, mask = this.get_conductances(mlats,mlts)
sh = np.ma.masked_array(sh,mask)
sp = np.ma.masked_array(sp,mask)
conmax = np.max([np.max(sh),np.max(sp)])

fig, axes = plt.subplots(6,1,figsize=(10,10),sharex=True)
fig.suptitle("Dawn-dusk meridian\n("+r"$v_{sw}=$"+str(vsw)+r", $B_y=$"+str(by)+r", $B_z=$"+str(bz)+r", $\Psi=$"+str(dptilt)+")",fontsize=15)

tickmlats = np.arange(50,131,10)
tickmlatswrap = tickmlats.copy()
tickmlatswrap[tickmlats > 90] = 180-tickmlatswrap[tickmlats > 90]

#potential
plt.sca(axes[0])
plt.plot(mlatsnowrap,pot)
plt.xticks(ticks=tickmlats,labels=tickmlatswrap)
#plt.title("Potential")
plt.ylabel("Potential [kV]")

#E-field
plt.sca(axes[1])
plt.plot(mlatsnowrap,Ed1)
plt.ylabel("$E_{d1}$ [mV/m]")

plt.sca(axes[2])
plt.plot(mlatsnowrap,Ed2)
plt.ylabel("$E_{d2}$ [mV/m]")

#EM work
plt.sca(axes[3])
plt.plot(mlatsnowrap,emwork)
plt.ylabel("EM work [mW/m²]")

#Conductances
plt.sca(axes[4])
plt.plot(mlatsnowrap,sh)
plt.ylabel("$\Sigma_H$ [mho]")
plt.ylim((0,conmax))

plt.sca(axes[5])
plt.plot(mlatsnowrap,sp)
plt.ylabel("$\Sigma_P$ [mho]")
plt.ylim((0,conmax))

plt.xlabel("MLat [deg]")

#Mark 90° MLat
for ax in axes:
    ax.axvline(90,linestyle='--',color='gray',zorder=-10)
    
plt.tight_layout()